A demonstration of the address cleaning process.

In [1]:
%matplotlib inline

In [2]:
import re
import matplotlib.pyplot as plt
import pandas as pd
from lovelyrita.data import read_data, column_map, summarize
from lovelyrita.clean import get_datetime, clean, impute_missing_times, drop_void
from lovelyrita.addresses import REPLACEMENTS

plt.style.use('seaborn')

# Load citations

In [3]:
data_paths = [
              "/data/lovely-rita/new/2012complete-output.csv",
              "/data/lovely-rita/new/2013complete-output.csv",
              "/data/lovely-rita/new/2014complete-output.csv",
              "/data/lovely-rita/new/2015complete-output.csv",
              "/data/lovely-rita/new/2016complete-output-2.csv"
             ]

In [4]:
citations = read_data(data_paths, clean=False)

In [ ]:
citations['street_name'] = None
citations['street_no'] = None
citations.drop(['street_suffix'], axis=1, inplace=True)

In [8]:
summarize(citations)

,Column Name,Data Type,Unique Count,Sample Value,null,% null
1,city,object,2,oakland,0,0.00
3,state,object,2,california,0,0.00
4,street,object,174339,6100 OLD QUARRY LOOP,0,0.00
0,badge_number,object,1462,937,1,0.00
7,ticket_issue_date,object,1828,1/1/12,1,0.00
9,ticket_number,object,1645813,838880801,1,0.00
10,violation_desc_long,object,88,OBEDIENCE TO SIGNS,1,0.00
11,violation_external_code,object,99,10.16.110,1,0.00
8,ticket_issue_time,object,2872,10:30:00,120,0.01
2,fine_amount,object,68,$73,409,0.02


# Parsing the street name

# What is the address format in the raw data?
Are addresses parsed into name and number? Or are they all inferred from `street`?

In any case, we can check database versions of those values with those parsed from the `street` field.

# Text replacements
Some things are easier to just replace at the outset. Use this strategy sparingly, since each replacement takes a lot of time.

In [9]:
for pattern, replacement in REPLACEMENTS:
    citations['street'].replace(pattern, replacement, regex=True, inplace=True)

## Lots
Many citations are in parking lots. Those will not have street numbers, so we should treat them separately. We will only concern ourselves with potential street addresses.

In [10]:
lot_indices = citations.street.str.contains('^[A-Z]LOT.*LOT$')
street_citations = citations.loc[~lot_indices]

## The most common pattern, 123 MAIN STREET

In [11]:
patt = re.compile(r'^(?P<street_no>\d+\-?\d?) (?P<street_name>[\w\d\s]+)')

In [12]:
street = street_citations.street.str.extract(patt, expand=True)
street.dropna(inplace=True)

In [13]:
citations.update(street)

In [14]:
summarize(citations)

,Column Name,Data Type,Unique Count,Sample Value,null,% null
1,city,object,2,oakland,0,0.00
3,state,object,2,california,0,0.00
4,street,object,174324,6100 OLD QUARRY LOOP,0,0.00
0,badge_number,object,1462,937,1,0.00
7,ticket_issue_date,object,1828,1/1/12,1,0.00
9,ticket_number,object,1645813,838880801,1,0.00
10,violation_desc_long,object,88,OBEDIENCE TO SIGNS,1,0.00
11,violation_external_code,object,99,10.16.110,1,0.00
8,ticket_issue_time,object,2872,10:30:00,120,0.01
2,fine_amount,object,68,$73,409,0.02


That covers over 98% of all citations.

In [15]:
citations.head()

,badge_number,city,fine_amount,state,street,street_name,street_no,ticket_issue_date,ticket_issue_time,ticket_number,violation_desc_long,violation_external_code
0,937,oakland,$73,california,6100 OLD QUARRY LOOP,OLD QUARRY LOOP,6100,1/1/12,10:30:00,838880801,OBEDIENCE TO SIGNS,10.16.110
1,8961,oakland,$263,california,1800 12TH AVE,12TH AVE,1800,1/1/12,12:00:00,838907806,COMM VEH IN RESIDENT,10.28.120
2,1855,oakland,$73,california,300 OAK COLISEUM,OAK COLISEUM,300,1/1/12,13:50:00,839915330,NO VALID REG,4000A
3,1071,oakland,$73,california,1 AIRPORT DRIVE,AIRPORT DRIVE,1,1/2/12,10:40:00,834934276,OBEDIENCE TO SIGNS,10.16.110
4,R1186,oakland,$73,california,1 AIRPORT DRIVE,AIRPORT DRIVE,1,1/2/12,16:00:00,839892207,OBEDIENCE TO SIGNS,10.16.110


## A less common pattern, P123-1 PARK STREET

In [16]:
patt = re.compile(r'^(?P<prefix>[A-Z]+)\-?(?P<street_no>\d+[\-\W]?\d?) '
                   '(?P<street_name>[\w\d\s]+)')
street = street_citations.street.str.extract(patt, expand=True)

In [17]:
street.dropna(inplace=True)

In [18]:
drop_indices = []
for i, s in street.iterrows():
    street_words = s.street_name.split(' ')
    drop = True
    for street_word in street_words:
        if street_word.startswith(s.prefix[0]):
            drop = False
    if drop:
        drop_indices.append(i)

In [19]:
street.drop(drop_indices, inplace=True)

In [20]:
citations.update(street)

In [21]:
lot_indices = citations.street.str.contains('^[A-Z]LOT.*LOT$')
street_citations = citations.loc[~lot_indices]

In [22]:
summarize(citations)

,Column Name,Data Type,Unique Count,Sample Value,null,% null
1,city,object,2,oakland,0,0.00
3,state,object,2,california,0,0.00
4,street,object,174324,6100 OLD QUARRY LOOP,0,0.00
0,badge_number,object,1462,937,1,0.00
7,ticket_issue_date,object,1828,1/1/12,1,0.00
9,ticket_number,object,1645813,838880801,1,0.00
10,violation_desc_long,object,88,OBEDIENCE TO SIGNS,1,0.00
11,violation_external_code,object,99,10.16.110,1,0.00
8,ticket_issue_time,object,2872,10:30:00,120,0.01
2,fine_amount,object,68,$73,409,0.02


# 99.2% successfully parsed
The rest are a mix of things that we should have parsed and things that actually contain no street number (parking lots, cross streets, etc.).

# All-in-one function

In [23]:
from lovelyrita.addresses import parse_addresses

In [24]:
street = parse_addresses(citations.street)

In [26]:
citations.update(street)

In [27]:
citations.head()

,badge_number,city,fine_amount,state,street,street_name,street_no,ticket_issue_date,ticket_issue_time,ticket_number,violation_desc_long,violation_external_code
0,937,oakland,$73,california,6100 OLD QUARRY LOOP,OLD QUARRY LOOP,6100,1/1/12,10:30:00,838880801,OBEDIENCE TO SIGNS,10.16.110
1,8961,oakland,$263,california,1800 12TH AVE,12TH AVE,1800,1/1/12,12:00:00,838907806,COMM VEH IN RESIDENT,10.28.120
2,1855,oakland,$73,california,300 OAK COLISEUM,OAK COLISEUM,300,1/1/12,13:50:00,839915330,NO VALID REG,4000A
3,1071,oakland,$73,california,1 AIRPORT DRIVE,AIRPORT DRIVE,1,1/2/12,10:40:00,834934276,OBEDIENCE TO SIGNS,10.16.110
4,R1186,oakland,$73,california,1 AIRPORT DRIVE,AIRPORT DRIVE,1,1/2/12,16:00:00,839892207,OBEDIENCE TO SIGNS,10.16.110
